In [1]:
import cv2
import numpy as np


In [2]:
def otsu(img):
    # 先将图像转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 计算直方图和总像素数
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    pixel_num = gray.shape[0] * gray.shape[1]
    
    # 初始化最大方差和阈值
    max_variance = 0
    threshold = 0
    
    # 遍历灰度级别计算类内方差和类间方差
    for i in range(256):
        p0 = np.sum(hist[:i+1]) / pixel_num
        p1 = np.sum(hist[i+1:]) / pixel_num
        if p0 == 0 or p1 == 0:
            continue
        w0 = np.sum(hist[:i+1]) / pixel_num
        w1 = np.sum(hist[i+1:]) / pixel_num
        u0 = np.sum(np.arange(0, i+1) * hist[:i+1]) / np.sum(hist[:i+1])
        u1 = np.sum(np.arange(i+1, 256) * hist[i+1:]) / np.sum(hist[i+1:])
        variance = w0 * w1 * (u0 - u1) ** 2
        if variance > max_variance:
            max_variance = variance
            threshold = i
    
    # 将阈值应用于原图像
    result = gray.copy()
    result[result > threshold] = 255
    result[result <= threshold] = 0
    
    return result

def iterative_threshold(img):
    # 先将图像转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 初始化阈值和上一次迭代的阈值
    threshold = 127
    last_threshold = -1
    
    # 不断迭代直到阈值稳定
    while threshold != last_threshold:
        last_threshold = threshold
        pixels_above = gray[gray > threshold]
        pixels_below = gray[gray <= threshold]
        mean_above = pixels_above.mean() if pixels_above.size > 0 else 0
        mean_below = pixels_below.mean() if pixels_below.size > 0 else 0
        threshold = int((mean_above + mean_below) / 2)
    
    # 将阈值应用于原图像
    result = gray.copy()
    result[result > threshold] = 255
    result[result <= threshold] = 0
    
    return result

# 读取测试图片
img = cv2.imread('test.jpg')

# 使用大津法进行阈值分割
result_otsu = otsu(img)

# 使用迭代法进行阈值分割
result_iterative = iterative_threshold(img)

# 显示结果
cv2.imshow('Original', img)
cv2.imshow('Otsu', result_otsu)
cv2.imshow('Iterative', result_iterative)
cv2.waitKey(0)
cv2.destroyAllWindows()
